# ETEO for Order Execution
This tutorial is to demonstrate an example of using ETEO to do order execution

## Step1: Import Packages

In [2]:
import warnings
warnings.filterwarnings("ignore")

import sys
from pathlib import Path
import os
import torch

ROOT = os.path.dirname(os.path.abspath("."))
sys.path.append(ROOT)

import argparse
import os.path as osp
from mmcv import Config
from trademaster.utils import replace_cfg_vals
from trademaster.nets.builder import build_net
from trademaster.environments.builder import build_environment
from trademaster.datasets.builder import build_dataset
from trademaster.agents.builder import build_agent
from trademaster.optimizers.builder import build_optimizer
from trademaster.losses.builder import build_loss
from trademaster.trainers.builder import build_trainer
from trademaster.transition.builder import build_transition
from collections import Counter

## Step2: Import Configs

In [3]:
parser = argparse.ArgumentParser(description='Download Alpaca Datasets')
parser.add_argument("--config", default=osp.join(ROOT, "configs", "order_execution",
                                                 "order_execution_BTC_eteo_eteo_adam_mse.py"),
                    help="download datasets config file path")
parser.add_argument("--task_name", type=str, default="train")
parser.add_argument("--test_style", type=str, default="-1")
args,_ = parser.parse_known_args()

cfg = Config.fromfile(args.config)
task_name = args.task_name
cfg = replace_cfg_vals(cfg)

## Step3: Build Dataset

In [4]:
dataset = build_dataset(cfg)

The 18_th segment length is less than the min length so it won't be tested


## Step4: Build Environment

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="train"))
valid_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="valid"))
test_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="test"))

## Step5: Build Net

In [6]:
action_dim = train_environment.action_dim
state_dim = train_environment.state_dim

cfg.act.update(dict(action_dim=action_dim, state_dim=state_dim))
cfg.cri.update(dict(action_dim=action_dim, state_dim=state_dim))

act = build_net(cfg.act)
cri = build_net(cfg.cri)

## Step6: Build Optimizer

In [7]:
act_optimizer = build_optimizer(cfg, default_args=dict(params=act.parameters()))
cri_optimizer = None

## Step7: Build Loss

In [8]:
criterion = build_loss(cfg)

## Step8: Build Transition

In [9]:
transition = build_transition(cfg)

## Step9: Build Agent

In [10]:
agent = build_agent(cfg, default_args=dict(action_dim=action_dim,
                                           state_dim=state_dim,
                                           act=act,
                                           cri=cri,
                                           act_optimizer=act_optimizer,
                                           cri_optimizer=cri_optimizer,
                                           criterion=criterion,
                                           transition=transition,
                                           device=device))

## Step10: Build Trainer

In [11]:
trainer = build_trainer(cfg, default_args=dict(train_environment=train_environment,
                                                       valid_environment=valid_environment,
                                                       test_environment=test_environment,
                                                       agent=agent,
                                                       device=device,
                                                       ))

| Arguments Keep work_dir: /Users/wentaozhang/workspace/RA/TradeMaster/work_dir/order_execution_BTC_eteo_eteo_adam_mse


## Step11: Train, Valid and Test

In [12]:
trainer.train_and_valid()
trainer.test()

Train Episode: [1/10]


KeyboardInterrupt: 